In [1]:
%load_ext autotime

In [1]:
import numpy as np
from bisect import bisect_left

In [3]:
from platform import python_version
print(python_version())

3.6.10


In [2]:
with open("ft_vectors_cbow_50_10_0_05.vec", "r") as f:
    res = dict()
    next(f)
    for line in f:
        skill = line.split(' ', 1)[0]
        vec = np.array(list(line.split(' ', 1)[1].splitlines()))
        res[skill] = vec

In [3]:
res.keys()

dict_keys(['team', 'development', 'business', '</s>', 'support', 'knowledge', 'software', 'data', 'technical', 'management', 'client', 'systems', 'based', 'solutions', 'including', 'design', 'projects', 'services', 'technology', 'environment', 'engineer', 'clients', 'project', 'ensure', 'provide', 'developer', 'manager', 'understanding', 'technologies', 'service', 'delivery', 'ability', 'develop', 'engineering', 'security', 'leading', 'teams', 'quality', 'training', 'customers', 'product', 'essential', 'experienced', 'system', 'deliver', 'responsible', 'help', 'products', 'process', 'global', 'communication', 'customer', 'organisation', 'developing', 'tools', 'able', 'use', 'manage', 'infrastructure', 'testing', 'processes', 'digital', 'responsibilities', 'build', 'agile', 'analysis', 'industry', 'offer', 'providing', 'java', 'network', 'office', 'managing', 'analyst', 'supporting', 'performance', 'test', 'staff', 'platform', 'sql', 'relevant', 'building', 'change', 'ensuring', 'commer

In [42]:
'buws' in 'business'

False

In [43]:
def word_exists(wordlist, word):
    return any(w.startswith(word) for w in wordlist)

In [51]:
type(list(res.keys()))

list

In [56]:
[word for word in res.keys() if word.startswith('bus')]

['business',
 'business_analyst',
 'businesses',
 'busy',
 'business_intelligence',
 'business_analysts',
 'business_for_supply_temporary_workers',
 'business_unit',
 'business_units',
 'business_cases',
 'business_relation',
 'business_acumen',
 'business_continuity',
 'business_as_sole_hiring_our_customer',
 'business_objects',
 'bus',
 'business_as_set_out_conduct',
 'businesses_regulations_it_is_a_within_allegis_group',
 'business_intelligence_analyst',
 'business_as_usual',
 'business_respect_temporary',
 'business_our_clients_these_behaviours',
 'business_intelligence_bi',
 'business_square_one_embraces',
 'business_logic',
 'business_an_as_defined_within_conduct',
 'business_when_providing_work_finding',
 'business_readiness',
 'business_order_to_identify',
 'businesses_regulations',
 'business_or_an_for_all_temporary_respectively',
 'business_specialising_within_areas',
 'business_relation_to_this_registered',
 'business_depending_precise',
 'business_can_receive_queens',
 'bus

In [55]:
bisect_left(list(res.keys()), 'bus')

14219

In [54]:
print(list(res.keys())[bisect_left(list(res.keys()), 'bus')].startswith('bus'))

#word_exists(res.keys(), 'bus')

False


In [5]:
res['business']

array(['4.7661 -0.48138 -2.064 2.4272 -1.8309 -4.5182 -0.67071 -0.19547 0.035257 1.9927 -0.2248 2.9225 1.5496 -0.55769 -0.034783 -1.694 0.12069 -1.7727 -0.47568 -0.56894 2.7884 -0.155 -2.1702 -2.3969 0.76749 3.1782 -2.1142 -0.17875 -1.3623 -5.8829 -1.8709 -1.2331 -2.681 -1.2428 3.7101 0.076153 1.2722 -1.8538 -2.1967 3.9081 1.1356 1.1152 3.4396 -5.8832 -1.282 2.5338 -0.20794 0.54071 -0.81548 4.3923 '],
      dtype='<U393')

In [56]:
import ipywidgets as widgets
import difflib

In [87]:
text = widgets.FloatText()
slider = widgets.FloatSlider()
display(text,slider)
mylink = widgets.jsdlink((text, 'value'), (slider, 'value'))

FloatText(value=0.0)

FloatSlider(value=0.0)

In [2]:
from IPython.display import display
import ipywidgets as widgets

a = widgets.IntSlider(value=3, min=0, max=5)
b = widgets.IntSlider(value=7, min=6, max=10)
def on_value_change(change):
    new_a = change['new']
    b.min = new_a + 1
    b.max = new_a + 6
a.observe(on_value_change, names='value')
display(a, b)

IntSlider(value=3, max=5)

IntSlider(value=7, max=10, min=6)

In [60]:
words = ['correct', 'horse', 'battery', 'staple']
items = [widgets.Button(description=w) for w in words]
widgets.HBox(items)

In [7]:
def findSubSting(word, words_list):
    return [w for w in sorted(words_list) if w.lower().startswith(word.lower())]

In [74]:

w = widgets.Textarea(
    value='',
    placeholder='Type something',
    description='Skill:',
    disabled=False,
    tooltip='Enter the name of the Text field'
)

display(w)

def print_text(widget):
    [word for word in res.keys() if word.startswith('bus')]
    old_input = widget['old']
    new_input = widget['new'].split(' ')[-1]
    # In questo modo però si cattura solo l'ultimo carattere inserito, trovare un modo più intelligente di 
    # dividere la sottoparola che si sta inserendo da quelle già inserite
    new_skill = ''.join([li.split('+ ')[-1] for li in difflib.ndiff(old_input, new_input) if li[0] != ' '])
    #print('new_skill', new_skill)
    #print('---------------')
    if len(new_input)>3:
        grammatical_suggests = findSubSting(new_input, res.keys())[:4]
        if grammatical_suggests:
            print(f"suggerimento: {grammatical_suggests}")
        else:
            print("Nessun suggerimento")
        
        btn_list = list()
        for sub in grammatical_suggests:
        
            b = widgets.Button(
                description=sub,
                disabled=False,
                button_style='', # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Click me'
            )
            
            b.on_click(my_event_handler)
            btn_list.append(b)
        display(widgets.HBox(btn_list))
        

w.observe(print_text, names='value')

Textarea(value='', description='Skill:', placeholder='Type something')

suggerimento: ['busi', 'busier', 'busiest', 'busin']


suggerimento: ['busin', 'busine', 'busines', 'business']


In [72]:
def my_event_handler(btn_object):
    print('You pressed the {} button!'.format(btn_object.description))
    skill_list = w.value.split(' ')
    skill_list[-1] = btn_object.description
    new_contest = ' '.join(skill_list)
    w.value = new_contest
    btn_object.close()

## Prove con il calcolo della matrice di similarità 

In [ ]:
import io
import fasttext
import numpy as np
import gensim
from gensim.models import Word2Vec, KeyedVectors

In [ ]:
# define training data
sentences = [['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec'],
             ['this', 'is', 'the', 'second', 'sentence'],
             ['yet', 'another', 'sentence'],
             ['one', 'more', 'sentence'],
             ['and', 'the', 'final', 'sentence']]
# train model
model = Word2Vec(sentences, min_count=1)
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
print(words)
# access vector for one word
print(model['sentence'])

In [ ]:
new_model.wv.syn0.T

In [ ]:
M = gensim.similarities.MatrixSimilarity(gensim.matutils.Dense2Corpus(new_model.wv.vectors))

In [ ]:
new_model[list(new_model.wv.vocab)][0]

In [ ]:
result = model.most_similar('first')
print(result)

In [ ]:

matrix = np.dot(model.wv.syn0norm, model.wv.syn0norm.T)

In [ ]:
from gensim.similarities import MatrixSimilarity

m2 = gensim.similarities.MatrixSimilarity(gensim.matutils.Dense2Corpus(model.wv.syn0))

In [ ]:
similarity_array[1,2]

In [ ]:
similarity_matrix = []
for sims in m2:
    similarity_matrix.append(sims)
similarity_array = np.array(similarity_matrix)

In [ ]:
aa = gensim.similarities.MatrixSimilarity(gensim.matutils.Dense2Corpus(model.wv.syn0.T))

In [ ]:
aa.get_similarities([(15, 15), (15, 15)])

In [ ]:
from scipy.spatial.distance import cdist
similarity =cdist(model.wv.syn0, model.wv.syn0, metric='cosine')

similarity # similarity between 'cat' and 'dog'